In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#read credit retirement data
credits=pd.read_csv('data/CDP/credits_2023.csv')
credits['credit_retired']=pd.to_numeric(credits['credit_retired'], errors='coerce')

#read company data
target_car=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_car')
target_og=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_OG')
target_airline=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_Airlines')

#read credit prices
credit_prices=pd.read_excel('data/company_data/Data.xlsx', sheet_name='credit_prices_ecosystem_marketp').set_index('Category')

#compile list of all companies
company_list=list(target_car['CDP_ID'].unique())+list(target_og['CDP_ID'].unique())+list(target_airline['CDP_ID'].unique())
credits=credits[credits['Account number'].isin(company_list)]
credits_compliance=credits[credits['purpose_retiring'].str.contains('ompliance')==True]

#filter for voluntary credits only
credits=credits[credits['purpose_retiring'].str.contains('oluntary')==True]
credits=credits[credits['purchased_issued'].str.contains('urchased')==True]

missing_credits=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Missing_Credits')


## Classify credits to match Ecosystem Marketplace pricing data

In [3]:
#transform creidt types stated in CDP to categories reported in Ecosystem Marketplace
project_type_dict = dict.fromkeys(credit_prices.index)

project_type_dict['Forestry & Land use'] = ['Afforestation', 'Reforestation','Other, please specify: REDD+','Other, please specify: Forest protection','Forest ecosystem restoration',
                                            'Other, please specify: Biodiversity Reserve Project','Peatland protection and restoration','Other, please specify: Forestry','Other, please specify: Forest Conservation']
project_type_dict['Renewable Energy'] = ['Solar', 'Wind', 'Hydro', 'Geothermal', 'Biomass energy','Landfill gas','Other, please specify: Forest Protection', 'Agroforestry',]
project_type_dict['Chemical Processing & Industrial Manufacturing'] = ['Other, please specify: Carbon capture and storage (CCS)','N2O',]
project_type_dict['Household Community Devices'] = ['Clean cookstove distribution','Energy efficiency: households','Other, please specify: Impact Carbon Global Safe Water','Community projects']
project_type_dict['Agriculture'] = ['Soil carbon sequestration']
project_type_dict['Waste Disposal'] =[]
project_type_dict['Transportation'] = ['Transport']
project_type_dict['Energy Efficiency Fuel Switching'] = ['Other, please specify: Waste Heat Recovery','Other, please specify: Energy Recovery','Energy efficiency: own generation', 'Other, please specify: Natural gas','Energy efficiency: supply side','Energy demand']

project_type_dict['Wooden Building Elements'] = ['Other, please specify: Wooden Building Elements']
project_type_dict['Biochar']=['Biochar']

project_type_dict['exclude']=[]
project_type_dict['mixture']=['Other, please specify: Mixture of project types', np.nan,'Other, please specify: 森林経営と省エネ活動','Other',]


inverted_dict = {}
# Inverting the dictionary
for key, values in project_type_dict.items():
    for value in values:
        inverted_dict[value] = key


credits['price']=np.nan

for index, rwos in credits.iterrows():
    project_type=credits.loc[index,'project_type']
    category=inverted_dict[project_type]
    credits['price'].loc[index]=credit_prices.loc[category,2022]

/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_27668/1255528650.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  credits['price'].loc[index]=credit_prices.loc[category,2022]
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ip

In [4]:
# transform creidt types of credits found in registries for companies in sample to categories reported in Ecosystem Marketplace
missing_credits['Project Type']=missing_credits['Project Type'].replace(np.nan,'avg')

project_type_dict = dict.fromkeys(credit_prices.index)

project_type_dict['Forestry & Land use'] = ['Agriculture Forestry and Other Land Use']
project_type_dict['Renewable Energy'] = ['Energy industries (renewable/non-renewable sources)']
project_type_dict['Chemical Processing & Industrial Manufacturing'] = []
project_type_dict['Household Community Devices'] = []
project_type_dict['Agriculture'] = []
project_type_dict['Waste Disposal'] =[]
project_type_dict['Transportation'] = []
project_type_dict['Energy Efficiency Fuel Switching'] = ['Energy demand']

project_type_dict['Wooden Building Elements'] = []
project_type_dict['Biochar']=[]

project_type_dict['exclude']=[]

project_type_dict['mixture']=['avg']



inverted_dict = {}
# Inverting the dictionary
for key, values in project_type_dict.items():
    print(key)
    if len(values)>0:
        for value in values:
            inverted_dict[value] = key


missing_credits['price']=np.nan

for index, rwos in missing_credits.iterrows():
    project_type=missing_credits.loc[index,'Project Type']
    category=inverted_dict[project_type]
    missing_credits['price'].loc[index]=credit_prices.loc[category,2022]

Forestry & Land use
Renewable Energy
Chemical Processing & Industrial Manufacturing
Household Community Devices
Energy Efficiency Fuel Switching
Waste Disposal
Agriculture
Transportation
Wooden Building Elements
Biochar
mixture
exclude


/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_27668/2575416819.py:38: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  missing_credits['price'].loc[index]=credit_prices.loc[category,2022]
/var/folders/y_/tm05xtp10jn9jjjr7109pxs0000

In [5]:
#calculate costs of missing credits
missing_credits['money_spent']=missing_credits['price']*missing_credits['Quantity Issued']
missing_credits.groupby('Company Name')['money_spent'].sum().sort_values(ascending=False)

Company Name
Deutsche Lufthansa                      3648150.00
Inpex Corporation                       1271959.78
Air France-KLM                           751740.00
Cathay Pacific Airways Limited           195047.05
Petroliam Nasional Berhad (PETRONAS)     158429.44
Gol Linhas Aereas Inteligentes S.A.       75770.97
Finnair                                   35508.66
Turkish Airlines                          32730.17
Grupo Aeromexico SAB de CV                30548.65
Wizz Air Holdings                          7281.56
Name: money_spent, dtype: float64

In [6]:
#calculate costs of credits for CDP reported companies
credits['money_spent']=credits['price']*credits['credit_retired']
missing_credits['money_spent']=missing_credits['price']*missing_credits['Quantity Issued']

In [7]:
credits

,Account number,Organization,Row,RowName,project_type,mitigatin_activity,project_description,credit_retired,purpose_retiring,purchased_issued,standard,price,money_spent
3,550,ANA Holdings Inc.,1,NaN,"Other, please specify: 森林経営と省エネ活動",Emissions reduction,当社グループは、2019年9月からキャノン製の複合機の利用を順次開始している。当該複合機は、...,152.0,Voluntary offsetting,Purchased,"Other private carbon crediting program, please...",7.37,1120.24
6,1932,BMW AG,1,NaN,Solar,Emissions reduction,Small project in renewable energies and a smal...,44835.0,Voluntary offsetting,Purchased,Gold Standard,4.16,186513.60
7,1932,BMW AG,2,NaN,Clean cookstove distribution,Emissions reduction,Efficient cookpots (60-80% more efficient than...,59500.0,Voluntary offsetting,Purchased,Gold Standard,8.55,508725.00
8,1932,BMW AG,3,NaN,Energy efficiency: households,Emissions reduction,"Small biogas units for households in rural, po...",440550.0,Voluntary offsetting,Purchased,Gold Standard,8.55,3766702.50
9,1932,BMW AG,4,NaN,Energy efficiency: households,Emissions reduction,"Small biogas units for households in rural, po...",306643.0,Voluntary offsetting,Purchased,Gold Standard,8.55,2621797.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,866803,Mercedes-Benz Group AG,1,NaN,"Other, please specify: Impact Carbon Global Sa...",Emissions reduction,The stated goal of the PoA is the widespread d...,400000.0,Voluntary offsetting,Purchased,Gold Standard,8.55,3420000.00
230,866803,Mercedes-Benz Group AG,2,NaN,Biomass energy,Emissions reduction,Wastewater Treatment and Biogas Utilization Pr...,12000.0,Voluntary offsetting,Purchased,Gold Standard,4.16,49920.00
231,866803,Mercedes-Benz Group AG,3,NaN,Geothermal,Emissions reduction,Geothermal power project in Indonesia,72996.0,Voluntary offsetting,Purchased,Gold Standard,4.16,303663.36
232,866803,Mercedes-Benz Group AG,4,NaN,Hydro,Emissions reduction,"Hydropower: small, low-impact Hydropower proje...",16500.0,Voluntary offsetting,Purchased,Gold Standard,4.16,68640.00


In [8]:
#save data
credits.to_pickle('data/temp/credits_prices.pkl')
missing_credits.to_pickle('data/temp/missing_credits_prices.pkl')